# Imports

In [30]:
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from pathlib import Path

In [31]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import numpy as np
import os, sys
import importlib
import yaml, json
from pathlib import Path
from sklearn.model_selection import train_test_split
#path append
sys.path.append(os.path.join(Path.cwd(), '../modules'))
import Evaluate, Helper, Present, Score, Synthetic, Modeling

In [32]:
#multiple prediction file load still works, just loaded a single file because too many things are being displayed otherwise.
config_file = '../config/config.yaml'
prediction_config_files = ['../config/predictions/pred1.yaml']
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

In [33]:
cwd_dir = Path.cwd() #For running locally
base_dir = cwd_dir.parent
data_dir= base_dir / 'data' /'simulated_health_data' / 'data'

# Create Partitioned Dataset and generate Train and Test Data

In [34]:
#read parquet file.
parquet_file = pq.ParquetFile( data_dir /'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.parquet')
#parquet_file.metadata
#parquet_file.schema

In [35]:
table = pq.read_table(data_dir /'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.parquet')

In [46]:
#Create a partioned parque file. 
partition_label = 'yyyymm'
train_date = '201701'
test_date = '201702'
if len(os.listdir(str(data_dir / 'dataset_part')))==0:
    pq.write_to_dataset(table, root_path= str(data_dir / 'dataset_part'), partition_cols=[partition_label])

In [37]:
dataset = ds.dataset(str(data_dir / 'dataset_part'), format="parquet", partitioning=[partition_label])
health_train = dataset.to_table(filter=ds.field(partition_label) == (partition_label+ '=' + train_date)).to_pandas()
health_test = dataset.to_table(filter=ds.field(partition_label) == (partition_label+ '=' + test_date)).to_pandas()
health_test

,person_id,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,cad8,...,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c,yyyymm
0,1000,1,1,1,1,1,1,1,1,0,...,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0,yyyymm=201702
1,1001,1,1,1,1,1,1,1,1,0,...,0.955057,0.190794,1.978757,2.605967,0.683509,0,0,-84.698542,1,yyyymm=201702
2,1002,1,1,1,1,1,1,1,1,0,...,0.390528,0.166905,0.184502,0.807706,0.072960,0,0,-0.915500,2,yyyymm=201702
3,1003,1,1,1,1,1,1,1,1,0,...,0.484752,-0.116773,1.901755,0.238127,1.996652,0,0,-51.293606,3,yyyymm=201702
4,1004,1,1,1,1,1,1,1,1,0,...,-0.156598,-0.031579,0.649826,2.154846,-0.610259,0,0,-62.943629,4,yyyymm=201702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,9995,1,1,1,0,0,0,0,0,0,...,0.389736,0.389634,-0.012625,0.161977,1.499623,0,0,-5.806354,95,yyyymm=201702
8996,9996,1,1,1,0,0,0,0,0,0,...,-0.265327,-0.424594,1.868926,1.437781,-1.389828,0,0,0.429037,96,yyyymm=201702
8997,9997,1,1,1,0,0,0,0,0,0,...,0.190861,0.432215,-0.731353,-0.075012,2.529188,0,0,-153.759155,97,yyyymm=201702
8998,9998,1,1,1,0,0,0,0,0,0,...,-0.903090,-0.713788,-0.805162,1.179985,-0.080986,0,0,-182.604320,98,yyyymm=201702


# Import Referral File for Matching

In [38]:
referral=Helper.read_file(directory=c_r['dir'],file=c_r['file'],file_format=c_r['file_format'],aws=c_r['aws'],bucket= c_r['bucket'])
#referral[c_r['columns']['id_column']] = referral[c_r['columns']['id_column']].astype(str)
referral[c_r['columns']['date_column']] = pd.to_datetime(referral[c_r['columns']['date_column']]).dt.strftime('%Y%m')
referral['target'] = pd.Series(np.ones(referral.shape[0], dtype=float))
referral = referral.pivot_table(index=c_r['columns']['id_column'], columns=c_r['columns']['date_column'], values='target', aggfunc='sum')
referral = referral.fillna(0)
referral

MYR,201602,201604,201605,201606,201607,201608,201609,201610,201611,201701,201704,201706,201708,201709,201711,201712
PERSON_ID,,,,,,,,,,,,,,,,
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
294,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Generate train and test label

In [39]:
from datetime import datetime

train_window = [0,6]

if(train_window[0]<0):
    start = datetime.strptime(train_date, '%Y%m') - pd.DateOffset(months=(-1)*train_window[0])
else:
    start = datetime.strptime(train_date, '%Y%m') + pd.DateOffset(months=train_window[0])

end = datetime.strptime(train_date, '%Y%m') + pd.DateOffset(months=train_window[1])

start = datetime.strftime(start, '%Y%m')
end = datetime.strftime(end, '%Y%m')

sl=slice(start,end)
train_label = referral.loc[:,sl] #also a shallow copy

#aggregate referrals
train_label = train_label.sum(axis=1)

#aggregated referral map to 1 if >1
train_label[train_label>1] = 1

#delete this line later -- for simulation purpose only
train_label.index = train_label.index * 10

train_label=train_label[train_label.index.isin(health_train['person_id'])]

assert (not(train_label.empty)), "Referral train_label df is empty after overlapping with prediction df"
 
train_label=train_label.append(pd.Series(0,index=set(health_train['person_id'].astype(int))-set(train_label.index.astype(int)))).sort_index()

train_label


1000    0.0
1001    0.0
1002    0.0
1003    0.0
1004    0.0
       ... 
9995    0.0
9996    0.0
9997    0.0
9998    0.0
9999    0.0
Length: 9000, dtype: float64

In [40]:
test_window = [0,6]

if(test_window[0]<0):
    start = datetime.strptime(test_date, '%Y%m') - pd.DateOffset(months=(-1)*test_window[0])
else:
    start = datetime.strptime(test_date, '%Y%m') + pd.DateOffset(months=test_window[0])

end = datetime.strptime(test_date, '%Y%m') + pd.DateOffset(months=test_window[1])

start = datetime.strftime(start, '%Y%m')
end = datetime.strftime(end, '%Y%m')

sl=slice(start,end)
test_label = referral.loc[:,sl] #also a shallow copy

#aggregate referrals
test_label = test_label.sum(axis=1)

#aggregated referral map to 1 if >1
test_label[test_label>1] = 1

#delete this line later -- for simulation purpose only
test_label.index = test_label.index * 10

test_label=test_label[test_label.index.isin(health_test['person_id'])]

assert (not(test_label.empty)), "Referral test_label df is empty after overlapping with prediction df"
 
test_label=test_label.append(pd.Series(0,index=set(health_test['person_id'].astype(int))-set(test_label.index.astype(int)))).sort_index()

test_label

1000    0.0
1001    0.0
1002    0.0
1003    0.0
1004    0.0
       ... 
9995    0.0
9996    0.0
9997    0.0
9998    0.0
9999    0.0
Length: 9000, dtype: float64

# Feature Extraction from Health file

In [41]:
train_X = health_train[['person_id','cad0', 'cad1', 'cad2', 'cad3', 'cad4', 'cad5',
       'cad6', 'cad7', 'cad8', 'cad9',  'dv0', 'dv1', 'dv2', 'dv3',
       'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9']]
test_X = health_test[['person_id','cad0', 'cad1', 'cad2', 'cad3', 'cad4', 'cad5',
       'cad6', 'cad7', 'cad8', 'cad9',  'dv0', 'dv1', 'dv2', 'dv3',
       'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9']]

train_X = train_X.set_index('person_id')
test_X = test_X.set_index('person_id')

train_y = train_label
test_y = test_label

print(train_X.shape)

(9000, 20)


In [43]:
train_predict, test_predict, trained_models = Modeling.perform_classification(train_X, test_X, train_y, test_y)

Fitting model:  k_nearest_neighbors_10
Fitting model:  svm_linear
Fitting model:  svm_nonlinear
Fitting model:  decision_tree
Fitting model:  naive_bayes
Fitting model:  random_forest
Fitting model:  adaboost
Fitting model:  quadratic_discriminant_analysis


In [44]:
test_predict

,target,k_nearest_neighbors_10,svm_linear,svm_nonlinear,decision_tree,naive_bayes,random_forest,adaboost,quadratic_discriminant_analysis
person_id,,,,,,,,,
1000,0.0,0.0,0.000016,0.000650,0.0,0.000000,0.0,0.144590,0.0
1001,0.0,0.0,0.000011,0.000588,0.0,0.000000,0.0,0.044705,0.0
1002,0.0,0.0,0.000012,0.000588,0.0,0.000000,0.0,0.020201,0.0
1003,0.0,0.0,0.000013,0.000587,0.0,0.000000,0.0,0.168417,0.0
1004,0.0,0.0,0.000012,0.000588,0.0,0.000000,0.0,0.023440,0.0
...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.000455,0.000580,0.0,0.952608,0.0,0.278602,0.0
9996,0.0,0.0,0.000558,0.000588,0.0,0.121642,0.0,0.047345,0.0
9997,0.0,0.0,0.000365,0.000588,0.0,0.905605,0.0,0.279575,0.0
